In [30]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque, defaultdict
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
import functools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 1 rows and 36 cols


## Part 1:
- Simple BFS using queue

In [17]:
stones = deque([int(x) for x in input_text[0].split()])

for it in tqdm(range(25)):
    for _ in range(len(stones)):
        s = stones.popleft()
        s_str = str(s)
        if s == 0:
            stones.append(1)
        elif len(s_str) % 2 == 0:
            stones.append(int(s_str[:len(s_str) // 2]))
            stones.append(int(s_str[len(s_str) // 2:]))
        else:
            stones.append(s * 2024)

len(stones)

100%|██████████| 25/25 [00:00<00:00, 202.35it/s]


183435

## part 2:
- Brute forcing the BFS is too slow
- Create graph (ignore all previously seen nodes)
- Keep count of visits in state table and do one graph pass per iteration

In [64]:
stones = deque([int(x) for x in input_text[0].split()])
mapping = defaultdict(list)

## create graph:
for it in tqdm(range(75)):
    for _ in range(len(stones)):
        s = stones.popleft()
        if s in mapping:
            continue 
        
        s_str = str(s)
        if s == 0:
            mapping[0] = [1]
        elif len(s_str) % 2 == 0:
            mapping[s] = [int(s_str[:len(s_str) // 2]), int(s_str[len(s_str) // 2:])]
        else:
            mapping[s] = [s * 2024]
        stones += mapping[s]

len(stones)

100%|██████████| 75/75 [00:00<00:00, 32621.88it/s]


0

In [66]:
stones = [int(x) for x in input_text[0].split()]

## two tables to flip between:
state_table_old  = {s: 0 for s in mapping.keys()}
state_table_new = {s: 0 for s in mapping.keys()}

for start_s in stones:
    state_table_old[start_s] = 1  # initiate

for it in range(75):
    for node_orig, nodes_dest in mapping.items():
        for n in nodes_dest:
            state_table_new[n] += state_table_old[node_orig]
    
    state_table_old = state_table_new
    state_table_new = {s: 0 for s in mapping.keys()}

sum(state_table_old.values())

218279375708592